# CKD Plan Designer Tutorial

## Install CKDPlanDesigner Package

In [6]:
! pip install -e project_code/

Obtaining file:///Users/nicholasdeveau/projects/bmi210/bmi210Project/project_code
  Preparing metadata (setup.py) ... done
  Attempting uninstall: CKDPlanDesigner
    Found existing installation: CKDPlanDesigner 0.0.0
    Uninstalling CKDPlanDesigner-0.0.0:
      Successfully uninstalled CKDPlanDesigner-0.0.0
  Running setup.py develop for CKDPlanDesigner


## Load Data
- This tutorial uses the freely available synthea example dataset
    - Download it here: https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip
    - unzip it to SYNTHEA_PATH
 

In [1]:
import pandas as pd
import os

from CKDPlanDesigner.utils import data_preprocessing as pre

SYNTHEA_PATH = '/Users/nicholasdeveau/projects/bmi210/bmi210Project_data/synthea_data/'

#### Here we build our patient dataframe
- This contains a vecor for each patient containing the required information for plan designing 

In [2]:
patients_df = pre.preprocess_synthea_data(SYNTHEA_PATH)

/Users/nicholasdeveau/projects/bmi210/bmi210Project/project_code/src/CKDPlanDesigner/utils/data_preprocessing.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_cond_df['VALUE'] = 1


In [3]:
patients_df

,patient_id,t2d,depression,bmi,hypertension,smoking,dob,race,ethnicity,gender,zip,eGFR,age
0,c8403116-d069-4130-aa02-39954fc971b5,True,False,False,False,False,1967-03-26,white,nonhispanic,F,NaN,66.9,55
1,73f124a4-e69e-4b8c-a447-e72d23dc54be,True,False,False,True,True,1967-06-14,white,nonhispanic,F,2170.0,70.5,55
2,31459cc4-45ed-4dee-b160-9f1a148a7724,False,False,True,False,False,1968-08-15,white,nonhispanic,M,NaN,75.4,54
3,799d0738-d5cd-4161-bdb8-e3c49e21ec66,False,False,True,True,True,1939-12-12,white,nonhispanic,M,2360.0,82.0,83
4,bae73dce-20fa-4a6a-8e46-b2ed541f9296,True,True,False,False,True,1958-04-27,white,nonhispanic,M,NaN,83.6,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,83c686df-0a1e-4950-9a2b-c84da98a2652,True,False,False,False,False,1962-01-24,asian,nonhispanic,F,NaN,81.7,60
112,9dfbf6ed-2378-4c27-9ff7-7e5c0619c236,True,False,True,False,False,1962-11-08,white,nonhispanic,F,NaN,88.6,60
113,1bacaab1-708b-42fe-8f12-fbe97a725440,True,False,True,False,False,1950-07-25,white,nonhispanic,F,2186.0,79.4,72
114,73fc00f8-a341-4f62-be27-07c35c3460bc,True,False,True,False,False,1941-05-04,white,nonhispanic,F,1027.0,84.8,81


## Working With a Patient

In [4]:
from CKDPlanDesigner.models import patients, plans, interventions
%load_ext autoreload
%autoreload patients
%autoreload plans
%autoreload interventions

In [5]:
patient1_config = patients_df.to_dict(orient='index')[5]

In [6]:
def get_age(date_str):
    try:
        delta = datetime.datetime.now() - datetime.datetime.strptime(str(date_str), '%Y-%M-%d')
        return delta.days // 365
    except:
        return 55 # average age

In [7]:
get_age(patient1_config['dob'])

55

In [25]:
patients_df[patients_df.age >= 75]

,patient_id,t2d,depression,bmi,hypertension,smoking,dob,race,ethnicity,gender,zip,eGFR,age
3,799d0738-d5cd-4161-bdb8-e3c49e21ec66,False,False,True,True,True,1939-12-12,white,nonhispanic,M,2360.0,82.0,83
5,53d371cc-eb68-4740-a68c-39707c6cb519,False,False,True,False,True,1941-03-25,asian,nonhispanic,M,NaN,71.0,81
6,9fe859fa-d775-494d-ae34-fa648e5d186d,False,False,True,True,True,1940-04-05,white,nonhispanic,F,NaN,84.8,82
12,a708bca4-8fc9-4567-8c22-8b13d44f4f45,True,False,False,False,False,1944-04-14,black,nonhispanic,M,NaN,23.1,78
19,baffdce5-d444-4c8c-a815-e7d00a01f7ca,False,False,True,True,False,1939-06-22,white,nonhispanic,F,2129.0,69.0,83
23,e4945ad4-5f21-460f-a420-98a32ef61d3d,False,False,True,False,True,1947-01-26,white,nonhispanic,M,NaN,81.7,75
25,9077ab04-92a1-4189-aa2b-93e50a5c32ae,True,False,True,True,False,1944-04-02,white,nonhispanic,F,1107.0,69.8,78
26,a7df9539-bc46-4527-928d-8f2911418ebd,False,False,True,False,True,1947-07-12,asian,nonhispanic,F,NaN,81.0,75
27,f95a8235-5c43-4926-84fb-391822847c30,True,False,True,True,True,1927-04-24,asian,nonhispanic,F,2176.0,65.2,95
28,4a757679-50a3-4392-8929-bec0876f4744,True,False,True,True,False,1937-09-07,white,nonhispanic,M,NaN,66.0,85


In [26]:
# patient_config = {'patient_id': 123,2
#                   'eGFR': 70,
#                   'depression': True
#                  }


# select the first patient and convert feature vector to a dictionary
# we pass this dictionary to the patient class on instantiation
patient1_config = patients_df.to_dict(orient='index')[0]
p1 = patients.Patient(**patient1_config)

patient2_config = patients_df.to_dict(orient='index')[44]
# patient2_config = patients_df.to_dict(orient='index')[15]
p2 = patients.Patient(**patient2_config)

#### Look at patient 1's charecteristics

In [27]:
p1.patient_config

{'patient_id': 'c8403116-d069-4130-aa02-39954fc971b5',
 't2d': True,
 'depression': False,
 'bmi': False,
 'hypertension': False,
 'smoking': False,
 'dob': '1967-03-26',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'F',
 'zip': nan,
 'eGFR': 66.9,
 'age': 55}

In [28]:
# we automatically assign a CKD stage when a patient is created
p1.stage

### Generate a care plan

In [29]:
p1.generate_careplan()
print(p1.careplan.plan_name)

Plan: Delay


In [30]:
p1.patient_config

{'patient_id': 'c8403116-d069-4130-aa02-39954fc971b5',
 't2d': True,
 'depression': False,
 'bmi': False,
 'hypertension': False,
 'smoking': False,
 'dob': '1967-03-26',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'F',
 'zip': nan,
 'eGFR': 66.9,
 'age': 55}

In [31]:
p1.careplan

In [32]:
p1.careplan.list_components()

Plan Components


[('T2D Management', 'Cost Reduction: 0.65')]

### Try it out for a different patient
- This patient has more co-morbidities. Check them out!
- Because of this, we should expect a more comprehensive care plan even though they're still just stage 2

In [33]:
import yaml

In [34]:
# yaml_path = os.path.join('/Users/nicholasdeveau/projects/bmi210/bmi210Project/project_code/src/CKDPlanDesigner/configs/ix_config.yaml')
# ix_config_yaml = open(yaml_path)
# ix_config = yaml.load(ix_config_yaml, Loader=yaml.FullLoader)

In [35]:
p2.patient_config

{'patient_id': 'c9942738-49f3-4459-818f-8da20072ff76',
 't2d': True,
 'depression': True,
 'bmi': False,
 'hypertension': True,
 'smoking': True,
 'dob': '1935-10-27',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'M',
 'zip': 2474.0,
 'eGFR': 9.5,
 'age': 87}

In [36]:
p2.stage

In [41]:
p2.generate_careplan()
p2.careplan.list_components(show_savings=False)

Plan Components


['ComfortAndPain']

In [42]:
p2.patient_config

{'patient_id': 'c9942738-49f3-4459-818f-8da20072ff76',
 't2d': True,
 'depression': True,
 'bmi': False,
 'hypertension': True,
 'smoking': True,
 'dob': '1935-10-27',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'M',
 'zip': 2474.0,
 'eGFR': 9.5,
 'age': 87}

In [43]:
p2.patient_config

{'patient_id': 'c9942738-49f3-4459-818f-8da20072ff76',
 't2d': True,
 'depression': True,
 'bmi': False,
 'hypertension': True,
 'smoking': True,
 'dob': '1935-10-27',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'M',
 'zip': 2474.0,
 'eGFR': 9.5,
 'age': 87}

In [45]:
p2.careplan.list_components()

Plan Components


[('ComfortAndPain', 'Cost Reduction: 1')]

In [47]:
p2.careplan.list_components(show_savings=False)

Plan Components


['ComfortAndPain']

In [ ]:
"""
1) profile patient (patient_config)
2) classified into disease (patient.stage)
3) unoptimized charecteristics (trey creating this in the ontology)
    3a) unoptimmized = some min number of "bad" criteria (from patient_config)
"""